In [186]:
# batch create items (with non-option) variations, categories from csv
# work flow


In [ ]:
# prerequiste squreup client package
# https://pypi.org/project/squareup/
# pip install squareup

In [187]:
## Add this to the first block in your note book to display json
# another option: 
import uuid
from IPython.core.display import display, HTML

import json

class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json_data
        self.uuid = str(uuid.uuid4())
        # This line is missed out in most of the versions of this script across the web, it is essential for this to work interleaved with print statements
        self._ipython_display_()
        
    def _ipython_display_(self):
        display(HTML('<div id="{}" style="height: auto; width:100%;"></div>'.format(self.uuid)))
        display(HTML("""<script>
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
          renderjson.set_show_to_level(1)
          document.getElementById('%s').appendChild(renderjson(%s))
        });</script>
        """ % (self.uuid, self.json_str)))

# Since this is copy-pasted wrongly(mostly) at a lot of places across the web, i'm putting the fixed, updated version here, mainly for self-reference


## To use this function, call this, this now works even when you have a print statement before or after the RenderJSON call
# RenderJSON(your_json)
# RenderJSON(json.loads(result.text))

In [188]:
import pandas as pd
import json

In [189]:
# module
# sandbox connection to "Square SandBox no2"

from square.client import Client
sandbox_token = "EAAAECCddSeDNsweJXdjNVux8FTfVgMUXAroCh7VcOuoPlgjYb7MrkULH8DZrZmJ"

client = Client(
    access_token = sandbox_token,
    environment='sandbox',
)


In [190]:
df = pd.read_excel("/home/hadoop/Desktop/SharedFolder/item_input_0526.xlsx")
df.head()

,item,name_item,description_item,name_variation,amount_variation,currency_variation
0,Shirt,Shirt,Shirt description,Small red shirt,2500,USD
1,NaN,Shirt,Shirt description,Large red shirt,3000,USD
2,hat,Hat,Hat description,Blu hat,1000,USD


In [191]:
import uuid
import pandas as pd

def reconstruct_body_variation(row):
    dict_row = row.to_dict()
    
    name_variation = dict_row.get("name_variation")
    id_variation = "#" + name_variation.replace(" ", "_")
    amount_variation = dict_row.get("amount_variation")
    currency_variation = dict_row.get("currency_variation")

    variation_dict = {
        "type": "ITEM_VARIATION",
        "id": id_variation,
        "item_variation_data": {
            "name": name_variation,
            "pricing_type": "FIXED_PRICING",
            "price_money": {
                "amount": amount_variation,
                "currency": currency_variation
            }
        }
    }
    return variation_dict


def body_constructor_for_item(df, uuid):
    df_item_id_descr = df[["name_item", "description_item"]].groupby("name_item").first().reset_index() #ugly, change later
    body_list = []
    for item_name, item_descr in df_item_id_descr.apply(lambda x: (x["name_item"], x["description_item"]), axis=1):
        name_item = item_name
        id_item = "#" + name_item.replace(" ", "_")
        description_item = item_descr

    #     construnct variation list
        df_body_item = df[df["name_item"] == name_item]
        variations_list = list(df_body_item.apply(lambda x : reconstruct_body_variation(x), axis=1).values)
    #     print(variations_list)

        body = {
        "idempotency_key": str(uuid.uuid4()),
        "object": {
            "type": "ITEM",
            "id": id_item,
            "item_data": {
                "name": name_item,
                "description": description_item,
                "variations": variations_list,
                "product_type": "REGULAR"
                    }
                }
            }
        body_list.append(body)
    return body_list


# test case
df_test = pd.DataFrame.from_dict({'name_item': {0: 'Shirt', 1: 'Shirt', 2: 'Hat'},
 'description_item': {0: 'Shirt description',
  1: 'Shirt description',
  2: 'Hat description'},
 'name_variation': {0: 'Small red shirt', 1: 'Large red shirt', 2: 'Blu hat'},
 'amount_variation': {0: 2500, 1: 3000, 2: 1000},
 'currency_variation': {0: 'USD', 1: 'USD', 2: 'USD'}})
body_list = body_constructor_for_item(df_test, uuid)
for body in body_list:
    RenderJSON(json.dumps(body))

In [217]:
# make create_item as a class

# import uuid
# import pandas as pd

class CreateItems:
    def __init__(self, dataframe):
        self.df = dataframe
        self.COLUMNS = ['name_item', 'description_item', 'name_variation', 
                           'amount_variation', 'currency_variation']
        self.COLUMN_REQUIRE = f"require columns: {self.COLUMNS}"
#         check columns
        for col in self.COLUMNS:
            if col not in self.df.columns:
                print(f"column \"{col}\" is missing from the dataframe")
                print(self.COLUMN_REQUIRE)
              
        
        
    def __reconstruct_body_variation(row):        
        dict_row = row.to_dict()

        name_variation = dict_row.get("name_variation")
        id_variation = "#" + name_variation.replace(" ", "_")
        amount_variation = dict_row.get("amount_variation")
        currency_variation = dict_row.get("currency_variation")

        variation_dict = {
            "type": "ITEM_VARIATION",
            "id": id_variation,
            "item_variation_data": {
                "name": name_variation,
                "pricing_type": "FIXED_PRICING",
                "price_money": {
                    "amount": amount_variation,
                    "currency": currency_variation
                }
            }
        }
        return variation_dict
        

    def body_constructor_for_item(self):
        try:
            df = self.df
            df_item_id_descr = df[["name_item", "description_item"]].groupby("name_item").first().reset_index() #ugly, change later
            body_list = []
            for item_name, item_descr in df_item_id_descr.apply(lambda x: (x["name_item"], x["description_item"]), axis=1):
                name_item = item_name
                id_item = "#" + name_item.replace(" ", "_")
                description_item = item_descr

            #     construnct variation list
                df_body_item = df[df["name_item"] == name_item]
                variations_list = list(df_body_item.apply(lambda x : reconstruct_body_variation(x), axis=1).values)
            #     print(variations_list)

                body = {
                "idempotency_key": str(uuid.uuid4()),
                "object": {
                    "type": "ITEM",
                    "id": id_item,
                    "item_data": {
                        "name": name_item,
                        "description": description_item,
                        "variations": variations_list,
                        "product_type": "REGULAR"
                            }
                        }
                    }
                body_list.append(body)
#             self.__body_list = body_list
            return body_list
        except Exception as e:
            print (e)
            
    def create_item_with_variations(self):
        reslut_list = []
        try:
            for body in self.body_constructor_for_item():
                result = client.catalog.upsert_catalog_object(body)
                reslut_list.append(result)
            return reslut_list
        except Exception as e:
            print(e, "\n", self.COLUMN_REQUIRE)
            
    
df_test = df.drop(["name_item"], axis=1)
object_create_item = CreateItems(df_test)
object_create_item.df
object_create_item.body_constructor_for_item()

object_create_item.create_item_with_variations()

column "name_item" is missing from the dataframe
require columns: ['name_item', 'description_item', 'name_variation', 'amount_variation', 'currency_variation']
"['name_item'] not in index"
"['name_item'] not in index"
'NoneType' object is not iterable 
 require columns: ['name_item', 'description_item', 'name_variation', 'amount_variation', 'currency_variation']


In [219]:
reslut_list = CreateItems(df).create_item_with_variations()
RenderJSON(json.loads(reslut_list[0].text))

In [133]:
# module upsert items with variatons (non-option type) into Square
# note: Square would NOT perform redundency check (so there will be multiple items with the same name)
# note: need to parse body as a dict inside a loop, (i.e., one item at a time)
for body in body_list:
    
    result = client.catalog.upsert_catalog_object(body)

    if result.is_success():
      RenderJSON(json.loads(result.text))
    elif result.is_error():
      print(result.errors)

In [35]:
# module
# list catalog to inspect the item library
result = client.catalog.list_catalog()

result = client.catalog.list_catalog(
  types = "ITEM"
)

if result.is_success():
  RenderJSON(json.loads(result.text))
elif result.is_error():
  print(result.errors)